In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot, VICO_loc
import plot as plot
import slices as slices
from ionfrac import VMIN, VMAX
import flux as flux
import ionfrac as ionfrac

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm

from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage

In [ ]:
saveloc = VICO_loc + '/pillowfiles/res_maps_fig'
v_mins = ionfrac.V_MINS
new_cmap = ionfrac.new_cmap

In [ ]:
freqs = np.array([.01, .05, .1, .5, 1, 5.3, 23, 43, 100, 230 ]) # GHz
freqs *= 10**9 # Hz

r_kpc = 1
heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
scales = heights_and_scales['scales'] # AU
heights = heights_and_scales['heights'] # AU  

colors = plot.COLORS

In [ ]:
rnums = np.array([9, 21, 39, 54])
ryears = np.array(['9,000 yrs', '21,000 yrs', '39,000 yrs', '54,000 yrs',])
rmasss = np.array([r'2 M$_\odot$', r'4 M$_\odot$', r'8 M$_\odot$', r'12 M$_\odot$',])
snaps_lr, snaps_mr, snaps_hr = flux.prep_res_snapshots()

mass_lr = np.array(['2 M$_\odot$ lr', '4 M$_\odot$ lr', '8 M$_\odot$ lr', '12 M$_\odot$ lr',])
mass_mr = np.array(['2 M$_\odot$ mr', '4 M$_\odot$ mr', '8 M$_\odot$ mr', '12 M$_\odot$ mr',])
mass_hr = np.array(['2 M$_\odot$ hr', '4 M$_\odot$ hr', '8 M$_\odot$ hr', '12 M$_\odot$ hr',])
# print(mass_lr, mass_mr, mass_hr)

In [ ]:
for snaps in [snaps_lr, snaps_mr, snaps_hr]:
    print(snaps[0].x1.shape, snaps[0].x2.shape)

In [ ]:
allsnaps = np.array([snaps_hr, snaps_mr, snaps_lr])
allsnaps = allsnaps.swapaxes(0,1).flatten()
for snap in allsnaps:
    print(snap.name)

In [ ]:
allmasss = np.array([r'2$M_\odot$'+'\nhi', r'2$M_\odot$'+'\nmed', r'2$M_\odot$'+'\nlo',
                         r'4$M_\odot$'+'\nhi', r'4$M_\odot$'+'\nmed', r'4$M_\odot$'+'\nlo', 
                         r'8$M_\odot$'+'\nhi', r'8$M_\odot$'+'\nmed', r'8$M_\odot$'+'\nlo',
                         r'12$M_\odot$'+'\nhi', r'12$M_\odot$'+'\nmed', r'12$M_\odot$'+'\nlo'])

allyears = np.array(['9,000 yrs hi-res', '9,000 yrs mid-res', '9,000 yrs lo-res',
                        '21,000 yrs hi-res', '21,000 yrs mid-res', '21,000 yrs lo-res',
                        '39,000 yrs hi-res', '39,000 yrs mid-res', '39,000 yrs lo-res',
                        '54,000 yrs hi-res', '54,000 yrs mid-res', '54,000 yrs lo-res'])

In [ ]:
for snap in allsnaps.flatten():
    ionfrac.load_average_ionfrac_array(snap)
    ionfrac.load_average_ionfrac_array_ratio(snap, const=False)
    snap.load_densities()
    snap.calculate_mids()
    snap.load_shock_variables()
    
# # Maximum Velocities for Contours
# for snap in allsnaps.flatten(): 
#     for v in ionfrac.v_mins:
#         ionfrac.max_velocity_proj_array(snap, v)

# Plotting Functions

##### *Slice Res Map Methods*

In [ ]:
def density_res_pcolormesh(Snap, year, scale, vmin, vmax, cmap='cividis', 
                        saveloc=False, show=False, show_cbar= False, 
                             show_xlabels=True, show_ylabels=True, cbar_pad=.2):
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('darkkhaki')

    cont = ax.pcolormesh(Snap.X2_1v, Snap.X1_2v, np.log10(np.rot90(Snap.dens[:,:,Snap.mid3])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
#     ax.plot(Snap.x2[9], Snap.x1[0], 
#             color='cyan', markersize=50, marker='+')
#     ax.plot(Snap.x2[9], Snap.x1[165], 
#             color='cyan', markersize=50, marker='+')
#     ax.plot(Snap.x2[270], Snap.x1[0], 
#             color='cyan', markersize=50, marker='+')
#     ax.plot(Snap.x2[270], Snap.x1[165], 
#             color='cyan', markersize=50, marker='+')
    
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
        
    if(show_cbar):
        cbar = fig.colorbar(cont, orientation = 'horizontal', 
                            fraction=0.038, pad=cbar_pad)
        cbar.set_label(label = (r'log($\rho$ / [g/cm^3])'), 
                       fontsize = 22,)
        cbar.ax.tick_params(rotation=45, labelsize = 19)
    ax.text(.02, .45, (r'$\mathbf{\rho}$'+'\n'+year), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 28, weight='bold', color='black')#, alpha=.5)
    filename = saveloc+'/'+Snap.name+'_0_density_'+str(scale)+'AU.png'
    if (saveloc != False): 
        fig.savefig(filename, bbox_inches='tight')
        plt.close(fig)
    return filename
        
        
def temp_res_pcolormesh(Snap, year, scale, vmin, vmax, cmap='hot', 
                        saveloc=False, show=False, show_cbar= False, 
                          show_xlabels=True, show_ylabels=True, cbar_pad=.2):
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')

    cont = ax.pcolormesh(Snap.X2_1v, Snap.X1_2v, np.log10(np.rot90(Snap.temperatures[:,:,Snap.mid3])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
        
    if(show_cbar):
        cbar = fig.colorbar(cont, orientation = 'horizontal', 
                            fraction=0.038, pad=cbar_pad)
        cbar.set_label(label = (r'log($T$ / [K])'), 
                       fontsize = 22)
        cbar.ax.tick_params(rotation=45, labelsize = 19)
    ax.text(.02, .45, (r'$\mathbf{T}$'+'\n'+year), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 28, weight='bold', color='white')#, alpha=.5)
    filename = saveloc+'/'+Snap.name+'_2_temp_'+str(scale)+'AU.png'
    if (saveloc != False): 
        fig.savefig(filename, bbox_inches='tight')
        plt.close(fig)
    return filename

      

##### *Ionfrac Res Map Methods*

In [ ]:
def mass_ionfrac_res_pcolormesh(Snap, year, scale, vmin, vmax, v, cmap=new_cmap,
                            saveloc=saveloc, show_cbar= False, 
                            show_xlabels=True, show_ylabels=True, 
                            cbar_pad=.2, show_freq=False, vertical_cbar=False, 
                            show_contours=False, show_legend=False):
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    v_min=v_mins[v]
            
    # Make projection log plots for snapshot
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')
    
    cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_mass[v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
    if(show_cbar):
        if(vertical_cbar):
            cbar = fig.colorbar(cont, orientation = 'vertical', 
                                fraction=0.038, pad=cbar_pad, extend='min')
            cbar.set_label(label = (r'log10($\langle\chi_\mathrm{H+}\rangle$ )'),
                          fontsize = 22) 
            cbar.ax.tick_params(rotation=45, labelsize = 19)
        else:
            cbar = fig.colorbar(cont, orientation = 'horizontal', 
                                fraction=0.038, pad=cbar_pad, extend='min')
            cbar.set_label(label = (r'log10($\langle\chi_\mathrm{H+}\rangle$ )'),
                          fontsize = 22) 
            cbar.ax.tick_params(rotation=45, labelsize = 19)
            
    if(show_contours):
        cntr = ax.contour(Snap.X2_1v , Snap.X1_2v, np.rot90(Snap.maxv1), 
                       levels = [1,10,100], colors = ['darkorange', 'red', 'magenta']) #linewidths = [1, 1.5, 1])
        h,_ = cntr.legend_elements()
        if(show_legend):
            ax.legend([h[0], h[1], h[2]], ['1 km/s', '10km/s', '100km/s'], loc = 'upper center', bbox_to_anchor=(.5, -.65))
    
#     ax.text(.02, .22, (r'$\mathbf{\langle\chi_{H+}\rangle, mass}$'+'\n'+year+'\n'+'$\mathbf{v_{min}}$='+str(v_min)+' km/s'), ha='left', va='top', 
#             transform=ax.transAxes, fontsize = 13, color='white', weight='bold')
    ax.text(.02, .45, (r'$\mathbf{\langle\chi_{H+}\rangle}$'+'\n'+year),
            ha='left', va='top', transform=ax.transAxes, 
            fontsize = 28, weight='bold', color='white')
    
    if (saveloc != False): 
        filename = saveloc+'/'+Snap.name+'_ionfrac_mass_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'
        fig.savefig(filename, bbox_inches='tight')
        return filename
    
def mass_ionfrac_res_const_pcolormesh(Snap, year, const, scale, vmin, vmax, v, cmap=new_cmap,
                            saveloc=saveloc, show_cbar= False, 
                            show_xlabels=True, show_ylabels=True, 
                            cbar_pad=.2, show_freq=False, vertical_cbar=False, 
                            show_contours=False, show_legend=False):
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    v_min=v_mins[v]
            
    # Make projection log plots for snapshot
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')
    
    if(const):
        cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_mass_const[v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    else: 
        cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_mass_ratio[v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
    if(show_cbar):
        if(vertical_cbar):
            cbar = fig.colorbar(cont, orientation = 'vertical', 
                                fraction=0.038, pad=cbar_pad, extend='min')
            cbar.set_label(label = (r'log10($\langle\chi_\mathrm{H+}\rangle$ )'),
                          fontsize = 22) 
            cbar.ax.tick_params(rotation=45, labelsize = 19)
        else:
            cbar = fig.colorbar(cont, orientation = 'horizontal', 
                                fraction=0.038, pad=cbar_pad, extend='min')
            cbar.set_label(label = (r'log10($\langle\chi_\mathrm{H+}\rangle$ )'),
                          fontsize = 22) 
            cbar.ax.tick_params(rotation=45, labelsize = 19)
            
    if(show_contours):
        cntr = ax.contour(Snap.X2_1v , Snap.X1_2v, np.rot90(Snap.maxv1), 
                       levels = [1,10,100], colors = ['darkorange', 'red', 'magenta']) #linewidths = [1, 1.5, 1])
        h,_ = cntr.legend_elements()
        if(show_legend):
            ax.legend([h[0], h[1], h[2]], ['1 km/s', '10km/s', '100km/s'], loc = 'upper center', bbox_to_anchor=(.5, -.65))
    
#     ax.text(.02, .22, (r'$\mathbf{\langle\chi_{H+}\rangle, mass}$'+'\n'+year+'\n'+'$\mathbf{v_{min}}$='+str(v_min)+' km/s'), ha='left', va='top', 
#             transform=ax.transAxes, fontsize = 13, color='white', weight='bold')
    ax.text(.02, .45, (r'$\mathbf{\langle\chi_{H+}\rangle}$'+'\n'+year),
            ha='left', va='top', transform=ax.transAxes, 
            fontsize = 28, weight='bold', color='white')
    ax.text(.65, .1, ('cooled'),
            ha='left', va='top', transform=ax.transAxes, 
            fontsize = 20, weight='bold', color='white')
    
    if ((saveloc) and (const)): 
        filename = saveloc+'/'+Snap.name+'_ionfrac_mass_const_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'             
        fig.savefig(filename, bbox_inches='tight')
        return filename
    elif ((saveloc)):  #  and (!const)
        filename = saveloc+'/'+Snap.name+'_ionfrac_mass_ratio_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'             
        fig.savefig(filename, bbox_inches='tight')
        return filename

##### *Intensity Proj Map Methods*

In [ ]:
freq_strings=np.array(['.01', '.05', '0.1', '0.5', '1.0', 
                       '5.3', '23', '43', '100', '230']) 

def intensity_res_pcolormesh(Snap, year, scale, f, vmin, vmax, cmap='CMRmap',
                        saveloc=False, show=False, show_cbar= False, show_xlabels=True, show_ylabels=True, cbar_pad=.2,
                             vertical_cbar=False):
    nu=freqs[f]
    Snap.load_intensity_variables(nu)
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')

    cont = ax.pcolormesh(Snap.X2_1v, Snap.X1_2v, np.log10(np.rot90(Snap.InuA[:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
        
    if(show_cbar):
        if(vertical_cbar):
            cbar = fig.colorbar(cont, orientation = 'vertical', 
                                fraction=0.038, pad=cbar_pad)
            cbar.set_label(label = (r'log($I_\nu$ / [mJy/as])'),
                          fontsize = 22) 
            cbar.ax.tick_params(rotation=45, labelsize = 19)
        else:
            cbar = fig.colorbar(cont, orientation = 'horizontal', 
                                fraction=0.038, pad=cbar_pad)
            cbar.set_label(label = (r'log($I_\nu$ / [mJy/as])'),
                          fontsize = 22) 
            cbar.ax.tick_params(rotation=45, labelsize= 16)
    ax.text(.02, .39, (freq_strings[f]+'GHz'), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 20, color='white', weight='bold')
    ax.text(.02, .45, ('\n'+year), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 28, color='white', weight='bold')
    if (saveloc != False): 
        filename = saveloc+'/'+Snap.name+'_'+str(nu/10**9)+'GHzIntensity_'+str(scale)+'AU.png' 
        fig.savefig(filename, bbox_inches='tight')
        return(filename)
        
def intensity_res_const_pcolormesh(Snap, year, const, scale, f, vmin, vmax, cmap='CMRmap',
                        saveloc=False, show=False, show_cbar= False, show_xlabels=True, show_ylabels=True, cbar_pad=.2,
                             vertical_cbar=False):
    nu=freqs[f]
    Snap.load_intensity_variables(nu)
    Snap.load_intensity_variables_const(nu=nu, const=const)
    
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')
    
    cont = ax.pcolormesh(Snap.X2_1v, Snap.X1_2v, np.log10(np.rot90(Snap.InuA_const[:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
        
    if(show_cbar):
        if(vertical_cbar):
            cbar = fig.colorbar(cont, orientation = 'vertical', 
                                fraction=0.038, pad=cbar_pad)
            cbar.set_label(label = (r'log($I_\nu$ / [mJy/as])'),
                          fontsize = 22) 
            cbar.ax.tick_params(rotation=45, labelsize = 19)
        else:
            cbar = fig.colorbar(cont, orientation = 'horizontal', 
                                fraction=0.038, pad=cbar_pad)
            cbar.set_label(label = (r'log($I_\nu$ / [mJy/as])'),
                          fontsize = 22) 
            cbar.ax.tick_params(rotation=45, labelsize = 19)
    ax.text(.02, .39, (freq_strings[f]+'GHz'), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 20, color='white', weight='bold')
    ax.text(.02, .45, ('\n'+year), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 28, color='white', weight='bold')
    ax.text(.65, .1, ('cooled'),
            ha='left', va='top', transform=ax.transAxes, 
            fontsize = 20, weight='bold', color='white')
    
    if ((saveloc) and (const)): 
        filename = saveloc+'/'+Snap.name+'_'+str(nu/10**9)+'GHzIntensity_const_'+str(scale)+'AU.png'             
        fig.savefig(filename, bbox_inches='tight')
        return filename
    elif ((saveloc)):  #  and (!const)
        filename = saveloc+'/'+Snap.name+'_'+str(nu/10**9)+'GHzIntensity_ratio_'+str(scale)+'AU.png'             
        fig.savefig(filename, bbox_inches='tight')
        return filename
    

In [ ]:
v_mins = np.array([1, 10, 100])
slicescales = np.array([4000, 25000])      
v = len(v_mins)-1

vmin_density, vmax_density = -22.1, -14.7    # 0 Density Slice
vmin_temp, vmax_temp = 0, 7                  # 1 Temperature Slice
vmin_ionfrac, vmax_ionfrac = -3, 0           # 2 Ionization Fraction Slice
vmin_intensity_noratio, vmax_intensity_noratio = -3, 2.8   
vmin_intensity_ratio, vmax_intensity_ratio = -4, 0    

def res_images(Snap, year, scale, saveloc = saveloc, 
                 show_cbar=False, xlabels=False, cbar_pad=.15):
    # density slice
    file0 = density_res_pcolormesh(Snap=Snap, year=year, scale = scale, 
                             vmin = vmin_density, vmax=vmax_density, saveloc=saveloc, 
                             cmap='cividis', show_cbar=show_cbar, 
                             show_xlabels=xlabel, show_ylabels=True,
                             cbar_pad=cbar_pad)
    # temperature slice
    file1 = temp_res_pcolormesh(Snap=Snap, year=year, scale = scale, 
                          vmin = vmin_temp, vmax=vmax_temp, saveloc=saveloc, 
                          show=True, cmap='hot', show_cbar=show_cbar, 
                          show_xlabels=xlabel, show_ylabels=False,
                          cbar_pad=cbar_pad)

    # Ionization Fraction noratio
    file2 = mass_ionfrac_res_pcolormesh(Snap, year, scale, v = v,
                            vmin=vmin_ionfrac, vmax = vmax_ionfrac, 
                            show_xlabels = xlabels, show_ylabels = False,
                            show_contours = False, show_legend = False, 
                            show_cbar = False, saveloc=saveloc)
    # 5.3 GHz Intensity noratio
    file3 = intensity_res_pcolormesh(Snap, year, scale, 5, 
                              vmin_intensity_noratio, vmax_intensity_noratio, 
                             show_xlabels = xlabels, show_ylabels = False, 
                              show_cbar=False, saveloc=saveloc)
    
    # Ionization Fraction ratio
    file4 = mass_ionfrac_res_const_pcolormesh(Snap, year, False, scale, v = v,
                            vmin=vmin_ionfrac, vmax = vmax_ionfrac, 
                            show_xlabels = xlabels, show_ylabels = False,
                            show_contours = False, show_legend = False, 
                            show_cbar = False, saveloc=saveloc)
    
    # 5.3 GHz Intensity
    file5 = intensity_res_const_pcolormesh(Snap, year, False, scale, 5, 
                              vmin_intensity_ratio, vmax_intensity_ratio, 
                             show_xlabels = xlabels, show_ylabels = False, 
                              show_cbar=False, saveloc=saveloc)
    
    return (file0, file1, file2, file3, file4, file5)


In [ ]:
ResSnapshots = np.array([snaps_hr, snaps_mr, snaps_lr])
ResSnapshots = np.swapaxes(ResSnapshots, 0, 1)


In [ ]:

import matplotlib.colors as colors
from obspy.imaging.cm import viridis_white_r 

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

cmap = plt.get_cmap('jet')
new_cmap = truncate_colormap(cmap, 0.1, 1)


def mass_ionfrac_res_pcolormesh(Snap, year, scale, vmin, vmax, v, cmap=new_cmap,
                            saveloc=saveloc, show_cbar= False, 
                            show_xlabels=True, show_ylabels=True, 
                            cbar_pad=.2, show_freq=False, vertical_cbar=False, 
                            show_contours=False, show_legend=False):
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    v_min=v_mins[v]
            
    # Make projection log plots for snapshot
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')
    
    cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_mass[v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
    if(show_cbar):
        if(vertical_cbar):
            cbar = fig.colorbar(cont, orientation = 'vertical', 
                                fraction=0.038, pad=cbar_pad, extend='min')
            cbar.set_label(label = (r'log10($\langle\chi_\mathrm{H+}\rangle$ )'),
                          fontsize = 22) 
            cbar.ax.tick_params(rotation=45, labelsize = 19)
        else:
            cbar = fig.colorbar(cont, orientation = 'horizontal', 
                                fraction=0.038, pad=cbar_pad, extend='min')
            cbar.set_label(label = (r'log10($\langle\chi_\mathrm{H+}\rangle$ )'),
                          fontsize = 22)
            cbar.ax.tick_params(rotation=45, labelsize = 19)
            
    if(show_contours):
        cntr = ax.contour(Snap.X2_1v , Snap.X1_2v, np.rot90(Snap.maxv1), 
                       levels = [1,10,100], colors = ['darkorange', 'red', 'magenta']) #linewidths = [1, 1.5, 1])
        h,_ = cntr.legend_elements()
        if(show_legend):
            ax.legend([h[0], h[1], h[2]], ['1 km/s', '10km/s', '100km/s'], loc = 'upper center', bbox_to_anchor=(.5, -.65))
    
#     ax.text(.02, .22, (r'$\mathbf{\langle\chi_{H+}\rangle, mass}$'+'\n'+year+'\n'+'$\mathbf{v_{min}}$='+str(v_min)+' km/s'), ha='left', va='top', 
#             transform=ax.transAxes, fontsize = 13, color='white', weight='bold')
    ax.text(.02, .45, (r'$\mathbf{\langle\chi_{H+}\rangle}$'+'\n'+year),
            ha='left', va='top', transform=ax.transAxes, 
            fontsize = 28, weight='bold', color='white')
    
    if (saveloc != False): 
        filename = saveloc+'/'+Snap.name+'_ionfrac_mass_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'
        fig.savefig(filename, bbox_inches='tight')
        plt.close(fig)
        return filename
    
def mass_ionfrac_res_const_pcolormesh(Snap, year, const, scale, vmin, vmax, v, cmap=new_cmap,
                            saveloc=saveloc, show_cbar= False, 
                            show_xlabels=True, show_ylabels=True, 
                            cbar_pad=.2, show_freq=False, vertical_cbar=False, 
                            show_contours=False, show_legend=False):
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    v_min=v_mins[v]
            
    # Make projection log plots for snapshot
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')
    
    if(const):
        cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_mass_const[v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    else: 
        cont = ax.pcolormesh(Snap.X2_1v , Snap.X1_2v, np.rot90(np.log10(Snap.ionfrac_mass_ratio[v,:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
    if(show_cbar):
        if(vertical_cbar):
            cbar = fig.colorbar(cont, orientation = 'vertical', 
                                fraction=0.038, pad=cbar_pad, extend='min')
            cbar.set_label(label = (r'log10($\langle\chi_\mathrm{H+}\rangle$ )'),
                          fontsize = 22) 
            cbar.ax.tick_params(rotation=45, labelsize = 19)
        else:
            cbar = fig.colorbar(cont, orientation = 'horizontal', 
                                fraction=0.038, pad=cbar_pad, extend='min')
            cbar.set_label(label = (r'log10($\langle\chi_\mathrm{H+}\rangle$ )'),
                          fontsize = 22)
            cbar.ax.tick_params(rotation=45, labelsize= 19)
            
    if(show_contours):
        cntr = ax.contour(Snap.X2_1v , Snap.X1_2v, np.rot90(Snap.maxv1), 
                       levels = [1,10,100], colors = ['darkorange', 'red', 'magenta']) #linewidths = [1, 1.5, 1])
        h,_ = cntr.legend_elements()
        if(show_legend):
            ax.legend([h[0], h[1], h[2]], ['1 km/s', '10km/s', '100km/s'], loc = 'upper center', bbox_to_anchor=(.5, -.65))
    
#     ax.text(.02, .22, (r'$\mathbf{\langle\chi_{H+}\rangle, mass}$'+'\n'+year+'\n'+'$\mathbf{v_{min}}$='+str(v_min)+' km/s'), ha='left', va='top', 
#             transform=ax.transAxes, fontsize = 13, color='white', weight='bold')
    ax.text(.02, .45, (r'$\mathbf{\langle\chi_{H+}\rangle}$'+'\n'+year),
            ha='left', va='top', transform=ax.transAxes, 
            fontsize = 28, weight='bold', color='white')
    ax.text(.65, .1, ('cooled'),
            ha='left', va='top', transform=ax.transAxes, 
            fontsize = 20, weight='bold', color='white')
    
    if ((saveloc) and (const)): 
        filename = saveloc+'/'+Snap.name+'_ionfrac_mass_const_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'             
        fig.savefig(filename, bbox_inches='tight')
        plt.close(fig)
        return filename
    elif ((saveloc)):  #  and (!const)
        filename = saveloc+'/'+Snap.name+'_ionfrac_mass_ratio_v'+str(v_mins[v])+'kms_'+str(scale)+'AU.png'             
        fig.savefig(filename, bbox_inches='tight')
        plt.close(fig)
        return filename

In [ ]:
freq_strings=np.array(['.01', '.05', '0.1', '0.5', '1.0', 
                       '5.3', '23', '43', '100', '230']) 

def intensity_res_pcolormesh(Snap, year, scale, f, vmin, vmax, cmap='CMRmap',
                        saveloc=False, show=False, show_cbar= False, show_xlabels=True, show_ylabels=True, cbar_pad=.2,
                             vertical_cbar=False):
    nu=freqs[f]
    Snap.load_intensity_variables(nu)
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')

    cont = ax.pcolormesh(Snap.X2_1v, Snap.X1_2v, np.log10(np.rot90(Snap.InuA[:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
        
    if(show_cbar):
        if(vertical_cbar):
            cbar = fig.colorbar(cont, orientation = 'vertical', 
                                fraction=0.038, pad=cbar_pad)
            cbar.set_label(label = (r'log($I_\nu$ / [mJy/as])'),
                          fontsize = 22) 
            cbar.ax.tick_params(rotation=45, labelsize = 19)
        else:
            cbar = fig.colorbar(cont, orientation = 'horizontal', 
                                fraction=0.038, pad=cbar_pad)
            cbar.set_label(label = (r'log($I_\nu$ / [mJy/as])'),
                          fontsize = 22)
            cbar.ax.tick_params(rotation=45, labelsize = 19)
    ax.text(.02, .39, (freq_strings[f]+'GHz'), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 20, color='white', weight='bold')
    ax.text(.02, .45, ('\n'+year), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 28, color='white', weight='bold')
    if (saveloc != False): 
        filename = saveloc+'/'+Snap.name+'_'+str(nu/10**9)+'GHzIntensity_'+str(scale)+'AU.png' 
        fig.savefig(filename, bbox_inches='tight')
        plt.close(fig)
        return(filename)
        
def intensity_res_const_pcolormesh(Snap, year, const, scale, f, vmin, vmax, cmap='CMRmap',
                        saveloc=False, show=False, show_cbar= False, show_xlabels=True, show_ylabels=True, cbar_pad=.2,
                             vertical_cbar=False):
    nu=freqs[f]
    Snap.load_intensity_variables(nu)
    Snap.load_intensity_variables_const(nu=nu, const=const)
    
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')
    
    cont = ax.pcolormesh(Snap.X2_1v, Snap.X1_2v, np.log10(np.rot90(Snap.InuA_const[:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('x (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('z (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
        
    if(show_cbar):
        if(vertical_cbar):
            cbar = fig.colorbar(cont, orientation = 'vertical', 
                                fraction=0.038, pad=cbar_pad)
            cbar.set_label(label = (r'log($I_\nu$ / [mJy/as])'),
                          fontsize = 22) 
            cbar.ax.tick_params(rotation=45, labelsize = 19)
        else:
            cbar = fig.colorbar(cont, orientation = 'horizontal', 
                                fraction=0.038, pad=cbar_pad)
            cbar.set_label(label = (r'log($I_\nu$ / [mJy/as])'),
                          fontsize = 22)
            cbar.ax.tick_params(rotation=45, labelsize = 19)
    ax.text(.02, .39, (freq_strings[f]+'GHz'), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 20, color='white', weight='bold')
    ax.text(.02, .45, ('\n'+year), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 28, color='white', weight='bold')
    ax.text(.65, .1, ('cooled'),
            ha='left', va='top', transform=ax.transAxes, 
            fontsize = 20, weight='bold', color='white')
    
    if ((saveloc) and (const)): 
        filename = saveloc+'/'+Snap.name+'_'+str(nu/10**9)+'GHzIntensity_const_'+str(scale)+'AU.png'             
        fig.savefig(filename, bbox_inches='tight')
        plt.close(fig)
        return filename
    elif ((saveloc)):  #  and (!const)
        filename = saveloc+'/'+Snap.name+'_'+str(nu/10**9)+'GHzIntensity_ratio_'+str(scale)+'AU.png'             
        fig.savefig(filename, bbox_inches='tight')
        plt.close(fig)
        return filename
    

In [ ]:
slicescales = np.array([4000, 25000])     
v_mins = np.array([1,10,100])
v = len(v_mins)-1

vmin_density, vmax_density = -22.1, -14.7    # 0 Density Slice
vmin_temp, vmax_temp = 0, 7                  # 1 Temperature Slice
vmin_ionfrac, vmax_ionfrac = -3, 0           # 2 Ionization Fraction Slice
vmin_intensity_noratio, vmax_intensity_noratio = -3, 2.8   
vmin_intensity_ratio, vmax_intensity_ratio = -4, 0   

# Colorbars

In [ ]:
cbar_snap = allsnaps[-1]
print(cbar_snap.name)

In [ ]:
print(saveloc)

In [ ]:
# use low res for this
cbar_snap = allsnaps[-1]
print(f"{cbar_snap.name=}")

s=0
scale = 25000
cname = density_res_pcolormesh(cbar_snap, ryears[s], scale, 
                         vmin=vmin_density, vmax=vmax_density, 
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc, cbar_pad=0.04)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_density.png', extracrop=0)
display(cbimg)

cname = temp_res_pcolormesh(cbar_snap, ryears[s], scale, 
                         vmin=vmin_temp, vmax=vmax_temp, 
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc, cbar_pad=0.04)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_temp.png', extracrop=0)
display(cbimg)

cname = mass_ionfrac_res_pcolormesh(cbar_snap, ryears[s], scale, 
                         vmin=vmin_ionfrac, vmax=vmax_ionfrac, v=0,
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc, cbar_pad=0.04)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_ionfrac_noratio.png', extracrop=0)
display(cbimg)


cname = intensity_res_pcolormesh(cbar_snap, ryears[s], scale, f=5,
                         vmin=vmin_intensity_noratio, vmax=vmax_intensity_noratio, 
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc, cbar_pad=0.04)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_intensity_noratio.png', extracrop=0)
display(cbimg)


cname = mass_ionfrac_res_const_pcolormesh(cbar_snap, ryears[s], False, scale, 
                         vmin=vmin_ionfrac, vmax=vmax_ionfrac, v=0,
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc, cbar_pad=0.06)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_ionfrac_ratio.png', extracrop=0)
display(cbimg)


cname = intensity_res_pcolormesh(cbar_snap, ryears[s], scale, f=5,
                         vmin=vmin_intensity_ratio, vmax=vmax_intensity_ratio, 
                         show_xlabels=False, show_ylabels=False, 
                         show_cbar=True, saveloc = saveloc, cbar_pad=0.04)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_intensity_ratio.png', extracrop=0)
display(cbimg)

# cbar_h_img = Image.open((saveloc+'/'+cbar_snap.name+'_0_density_'+str(scale)+'AU.png'))
# print(cbar_h_img.size)
# width, height = cbar_h_img.size
# left = 0
# right = width
# top = 200
# bottom = height
# cbar_h_img = cbar_h_img.crop((left, top, right, bottom))
# print(cbar_h_img.size)
# cbar_h_filename = saveloc+'/cbar_density.png'
# cbar_h_img.save(cbar_h_filename)
# display(cbar_h_img)

In [ ]:
 

def res_images(Snap, year, scale, saveloc = saveloc, 
                 show_cbar=False, xlabels=False, cbar_pad=.15):
    # density slice
    file0 = density_res_pcolormesh(Snap=Snap, year=year, scale = scale, 
                             vmin = vmin_density, vmax=vmax_density, saveloc=saveloc, 
                             cmap='cividis', show_cbar=show_cbar, 
                             show_xlabels=xlabel, show_ylabels=True,
                             cbar_pad=cbar_pad)
    # temperature slice
    file1 = temp_res_pcolormesh(Snap=Snap, year=year, scale = scale, 
                          vmin = vmin_temp, vmax=vmax_temp, saveloc=saveloc, 
                          show=True, cmap='hot', show_cbar=show_cbar, 
                          show_xlabels=xlabel, show_ylabels=False,
                          cbar_pad=cbar_pad)

    # Ionization Fraction noratio
    file2 = mass_ionfrac_res_pcolormesh(Snap, year, scale, v = v,
                            vmin=vmin_ionfrac, vmax = vmax_ionfrac, 
                            show_xlabels = xlabels, show_ylabels = False,
                            show_contours = False, show_legend = False, 
                            show_cbar = False, saveloc=saveloc)
    # 5.3 GHz Intensity noratio
    file3 = intensity_res_pcolormesh(Snap, year, scale, 5, 
                              vmin_intensity_noratio, vmax_intensity_noratio, 
                             show_xlabels = xlabels, show_ylabels = False, 
                              show_cbar=False, saveloc=saveloc)
    
    # Ionization Fraction ratio
    file4 = mass_ionfrac_res_const_pcolormesh(Snap, year, False, scale, v = v,
                            vmin=vmin_ionfrac, vmax = vmax_ionfrac, 
                            show_xlabels = xlabels, show_ylabels = False,
                            show_contours = False, show_legend = False, 
                            show_cbar = False, saveloc=saveloc)
    
    # 5.3 GHz Intensity
    file5 = intensity_res_const_pcolormesh(Snap, year, False, scale, 5, 
                              vmin_intensity_ratio, vmax_intensity_ratio, 
                             show_xlabels = xlabels, show_ylabels = False, 
                              show_cbar=False, saveloc=saveloc)
    
    return (file0, file1, file2, file3, file4, file5)


In [ ]:
AllResSnapshots = allsnaps.flatten()
AllResYears = allyears
AllResMasses = allmasss

In [ ]:
nrows = len(AllResSnapshots)
ncols = 6
scale = 4000
allresfiles = np.empty((nrows, ncols), dtype=object)
r=0
for s in range(0,len(AllResSnapshots)):
    # print(AllResYears[s])
    if(s < len(AllResSnapshots)-1):
        xlabel = False
    else: xlabel = True
    file0, file1, file2, file3, file4, file5 = \
        res_images(AllResSnapshots[s], AllResMasses[s], 
                   scale, saveloc = saveloc, 
                   show_cbar=False, xlabels=xlabel, 
                   cbar_pad=.2)
    
    allresfiles[r,0] = file0
    allresfiles[r,1] = file1
    allresfiles[r,2] = file2
    allresfiles[r,3] = file3
    allresfiles[r,4] = file4
    allresfiles[r,5] = file5
    r+=1

allresfiles_4000au = allresfiles

In [ ]:
allimage, details = plot.pil_image(allresfiles_4000au)
# display(allimage)
allimage = plot.pil_res_cbar(allimage, details, saveloc)
display(allimage)
allimage.save(VICO_loc+'/figures/res_maps/res_maps_4000au.png')

# 25000 au

In [ ]:
nrows = len(AllResSnapshots)
ncols = 6
scale = 25000
allresfiles = np.empty((nrows, ncols), dtype=object)
r=0
for s in range(0,len(AllResSnapshots)):
    # print(AllResYears[s])
    if(s < len(AllResSnapshots)-1):
        xlabel = False
    else: xlabel = True
    file0, file1, file2, file3, file4, file5 = \
        res_images(AllResSnapshots[s], AllResMasses[s], 
                   scale, saveloc = saveloc, 
                   show_cbar=False, xlabels=xlabel, 
                   cbar_pad=.2)
    
    allresfiles[r,0] = file0
    allresfiles[r,1] = file1
    allresfiles[r,2] = file2
    allresfiles[r,3] = file3
    allresfiles[r,4] = file4
    allresfiles[r,5] = file5
    r+=1

allresfiles_25000au = allresfiles

In [ ]:
allimage, details = plot.pil_image(allresfiles_25000au)
# display(allimage)
allimage = plot.pil_res_cbar(allimage, details, saveloc)
display(allimage)
allimage.save(VICO_loc+'/figures/res_maps/res_maps_25000au.png')